# Adatelemzés és adatvizualizáció mérnököknek - BMEVIMIAV28 2024/25/1
## Házifeladat Dokumentáció
### Harsányi Zsolt - XORZVV - NYC Crash Analysis

### Felhasznált adatkészletek

1. NYC Motor Vehicle Collisions
2. NYC Weather - 2016 to 2022

Feladatként én egy Dashboard-ot készítettem amelynek Github linkje az alábbi címen érhető el:
https://github.com/harsnyi/NYC-Crash-Analysis


Emellett a zip-ben mindkét adatkészletből elérhető egy mintavételezett verzió (100 minta)

### 1. NYC Motor Vehicle Collisions

In [1]:
import pandas as pd
df_crashed = pd.read_csv("NYC_motor_vehicle_collisions_sampled.csv")

A df rengeteg üres Borough értéket tartalmazott, ami általában ha üres volt, akkor egy csomó minden más is üres volt a soron, emiatt ezeket a sorokat eldobtam.

In [2]:
df_crashed = df_crashed[df_crashed["BOROUGH"].notnull()]

Ezután létrehoztam pár segédoszlopot hogy könnyebben tudjak majd aggregálni stb...

In [3]:
df_crashed["CRASH DATE"] = pd.to_datetime(df_crashed["CRASH DATE"], format='%m/%d/%Y')

In [4]:
df_crashed["CRASH TIME"] = pd.to_datetime(df_crashed["CRASH TIME"], format='%H:%M')

In [5]:
df_crashed["HOUR"] = df_crashed["CRASH TIME"].dt.hour
df_crashed["MINUTE"] = df_crashed["CRASH TIME"].dt.minute
df_crashed['YEAR'] = df_crashed['CRASH DATE'].dt.year
df_crashed['MONTH'] = df_crashed['CRASH DATE'].dt.month
df_crashed['DAY'] = df_crashed['CRASH DATE'].dt.day

In [6]:
df_crashed = df_crashed.drop(columns=['CRASH DATE', 'CRASH TIME'])

Utoljára eldobtam a 2012-es és 2024-es évet ugyanis ezek nem fednek le minden hónapot.

In [7]:
df_crashed = df_crashed[df_crashed['YEAR'] != 2012]
df_crashed = df_crashed[df_crashed['YEAR'] != 2024]

### 2. NYC Weather - 2016 to 2022

Ezután importáltam a Weather adatkészletet amit utána illesztettem a másik adatkészlethez, a vizualizációk során pedig a kombinált adatkészletet használtam.

In [8]:
df_weather = pd.read_csv("NYC_weather_sampled.csv")

In [9]:
df_weather['time'] = pd.to_datetime(df_weather['time'], errors='coerce')

In [10]:
df_crashed['crash_time'] = pd.to_datetime(df_crashed[['YEAR', 'MONTH', 'DAY', 'HOUR']])

In [11]:
df_weather = df_weather.sort_values('time')
df_crashed = df_crashed.sort_values('crash_time')

df_combined = pd.merge_asof(
    df_crashed,
    df_weather,
    left_on='crash_time',
    right_on='time',
    direction='backward'
)

In [12]:
df_combined.head()

,Unnamed: 0_x,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,1804431,BROOKLYN,11221.0,40.699628,-73.923832,"(40.699628, -73.9238321)",DE KALB AVENUE,WILSON AVENUE,NaN,0.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1796863,QUEENS,11354.0,40.764350,-73.825943,"(40.7643501, -73.8259432)",NORTHERN BOULEVARD,BOWNE STREET,NaN,0.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1773403,BROOKLYN,11232.0,40.651834,-74.003326,"(40.6518337, -74.0033264)",5 AVENUE,38 STREET,NaN,0.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1653595,MANHATTAN,10024.0,40.784355,-73.981165,"(40.7843547, -73.9811649)",WEST 79 STREET,WEST END AVENUE,NaN,0.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1621547,MANHATTAN,10022.0,40.758308,-73.962926,"(40.7583082, -73.9629257)",EAST 57 STREET,1 AVENUE,NaN,0.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dashboard

Ezután kezdtem el dolgozni a Dashboard-on amit a Plotly segítségével fejlesztettem Python nyelven.

Itt az alábbi vizualizációkat készítettem:
1. New York térkép amelyre minden évből lehet mintavételezni 1000 adatpontot: Ennek a lényege hogy könnyen értelmezhető vizualizációt adjon a New York-ban található városrészekről. A Latitude-ok alapján tettem fel a térképre az adatpontokat Geopandas segítségével.
2. Incidensek eloszlása 24 órára bontva
3. Incidensek eloszlása városi negyedekre bontva
4. Járművek száma balesetek szerint
5. Sérülések és halálesetek eloszlása városi negyedekre bontva
6. Balesetek és lehullott csapadék szerinti összefüggés

### Megválaszolt kérdések
- A Nap korai szakaszában jóval kevesebb incidens történt, mint a délutáni nyüzsgős időszakban
- Az esetek majdnem harmada Brooklynban történt
- Legtöbbször 2 autó ütközött
- Elenyésző a haláleset a sebesülésekkel szemben
- az ütközések úgy korrelálnak a csapadékmennyiséggel hogy egy bizonyos csapadékmennyiség fölött már jóval több ütközés volt.